# LoL Stats Visualization

In [ ]:
import backend.downloader as dl
import backend.assets as assets
import backend.db as db
import pandas as pd
import altair as alt

## Download data

In [ ]:
SUMMONERS = ["SUMMONER_NAME"]

dl.get_data(SUMMONERS, mode="update")

In [ ]:
assets.update_all()

## Get data from DBs

In [ ]:
# Read from DB to dataframe
NORMALIZED_DB_PATH = "./backend/db/data_normalized.db"

nconn = db.create_connection(NORMALIZED_DB_PATH)
md = db.read_full_table(nconn, "match_details")
t = db.read_full_table(nconn, "teams")
p = db.read_full_table(nconn, "participants")
pi = db.read_full_table(nconn, "participant_identities")
db.close_connection(nconn)

In [ ]:
# Add queue and map info
ASSETS_PATH = "./backend/db/assets.db"

aconn = db.create_connection(ASSETS_PATH)
qs = db.read_full_table(aconn, "queues")
db.close_connection(aconn)

## Match details exploration
### Create additional variables 

In [ ]:
# Get date from gameCreation
md["datetime"] = pd.to_datetime(md["gameCreation"], unit="ms")

# Get approximate match duration
md["gameDurationMinutes"] = round(md["gameDuration"] / 60, 0)
md["gameDurationMinutes"] = md["gameDurationMinutes"].astype("int16")

# Add queue names - NEXUSBLITZ games may be lost as their queue seems to not exists in the queues dataset
if not "queue" in md.columns:
    md = pd.merge(md, qs, on="queueId")

md

### Basic count charts

#### gameMode

In [ ]:
common_args = [
    alt.X("count():Q", axis=alt.Axis(grid=True, ticks=True)),
]

mode_text = alt.Chart().mark_text(dx=10).encode(
    *common_args,
    alt.Y("gameMode:N"),
    alt.Text("count():Q"),
)

mode_counts = alt.Chart().mark_bar().encode(
    *common_args,
    alt.Y("gameMode:N"),
    alt.Color("datetime:N", timeUnit="year"),
    alt.Tooltip("count():Q")
)

alt.layer(mode_counts, mode_text, data=md)

In [ ]:
alt.layer(mode_counts, mode_text, data=md).facet(
    row=alt.Row("datetime:N", timeUnit="yearmonth")
)

In [ ]:
alt.Chart(md).mark_tick(width=40, thickness=3).encode(
    alt.X("gameMode:N"),
    alt.Y("gameDurationMinutes:Q"),
    alt.Color("count():Q"),
    tooltip=[alt.Tooltip("count():Q"), alt.Tooltip("gameDurationMinutes:Q")]
).configure_view(width=350, height=350)

In [ ]:
alt.Chart(md).transform_density(
    'gameDurationMinutes',
    as_=['gameDurationMinutes', 'density'],
    groupby=['gameMode']
).mark_area(orient='horizontal').encode(
    y='gameDurationMinutes:Q',
    color='gameMode:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'gameMode:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
        ),
    )
).configure_facet(
    spacing=0
)

#### gameDurationMinutes

In [ ]:
duration_text = alt.Chart().mark_text(dx=10).encode(
    *common_args,
    alt.Y("gameDurationMinutes:N"),
    alt.Text("count():Q"),
)

duration_counts = alt.Chart().mark_bar().encode(
    *common_args,
    alt.Y("gameDurationMinutes:N"),
    alt.Color("datetime:N", timeUnit="year"),
    alt.Tooltip("count():Q")
)

alt.layer(duration_counts, duration_text, data=md)

In [ ]:
duration_counts = duration_counts.encode(
    alt.Color("gameMode:N")
)

alt.layer(duration_counts, duration_text, data=md)

#### datetime

In [ ]:
def datetime_layer_chart(time_unit):
    datetime_text = alt.Chart().mark_text(dx=10).encode(
        *common_args,
        alt.Text("count():Q"),
        alt.Y("datetime:N", timeUnit=time_unit)
    )

    datetime_counts = alt.Chart().mark_bar().encode(
        *common_args,
        alt.Y("datetime:N", timeUnit=time_unit)
    )

    return alt.layer(datetime_counts, datetime_text, data=md)

datetime_layer_chart("year")

In [ ]:
datetime_layer_chart("yearmonth")

In [ ]:
datetime_layer_chart("day")

In [ ]:
datetime_layer_chart("hours")

In [ ]:
alt.Chart(md).mark_circle().encode(
    alt.X("datetime:T", timeUnit="hoursminutes"),
    alt.Y("gameDurationMinutes:Q"),
    #alt.Color("datetime:N", timeUnit="day"),
    #alt.Size("count()"),
    tooltip=[alt.Tooltip("datetime:T", timeUnit="hoursminutes"),
        alt.Tooltip("gameDurationMinutes:Q")],
)

In [ ]:
hm = alt.Chart(md).mark_rect().encode(
    alt.X("datetime:O", timeUnit="day"),
    alt.Y("gameMode:O"),
    alt.Color("count()")
)

text = alt.Chart(md).mark_text().encode(
    alt.X("datetime:O", timeUnit="day"),
    alt.Y("gameMode:O"),
    alt.Text("count()")
)

(hm + text).configure_view(width=400, height=200)

### Maps

In [ ]:
alt.Chart(md).mark_bar().encode(
    alt.X("map:N"),
    alt.Y("count():Q")
).configure_view(width=200)

In [ ]:
lines = alt.Chart(md).mark_line(point=True).encode(
    alt.X("datetime:T", timeUnit="yearmonth"),
    alt.Y("count():Q"),
    alt.Color("map:N"),
    alt.Tooltip("count():Q")
) 
bars = alt.Chart(md).transform_calculate(diff="datum.map == 'Howling Abyss' ? 1 : -1").mark_bar().encode(
    alt.X("datetime:T", timeUnit="yearmonth"),
    alt.Y("sum(diff):Q"),
    alt.Tooltip("sum(diff):Q")
)

lines & bars

#### Queues

In [ ]:
alt.Chart(md).mark_circle(size=50).encode(
    alt.Y("queue:N"),
    alt.X("gameDurationMinutes:Q"),
    alt.Color("count():Q"),
    alt.Tooltip("count():Q")
).configure_view(height=300, width=500)

In [ ]:
squares = alt.Chart(md).mark_rect().encode(
    alt.X("datetime:N", timeUnit="day"),
    alt.Y("queue:N"),
    alt.Color("count():Q"),
    alt.Tooltip("count():Q"),
)

text = alt.Chart(md).mark_text().encode(
    alt.X("datetime:N", timeUnit="day"),
    alt.Y("queue:N"),
    alt.Text("count():Q")
)

(squares + text).configure_view(height=300, width=350)

## Participants exploration

In [ ]:
TBC